# Compiling all data sources

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
import time
opensecretsapi = os.getenv('opensecretsapi')
congressapi = os.getenv('congressapi')
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

## https://theunitedstates.io/: Members

In [5]:
url = 'https://theunitedstates.io/congress-legislators/legislators-current.json'
r = requests.get(url, headers=headers)
myjson = json.loads(r.text)
usio_members = pd.json_normalize(myjson)
usio_members.columns = usio_members.columns.str.replace('id.', '')
usio_members.to_csv('Data/usio_members.csv', index=False)

## Congress API: members

In [6]:
root = 'https://api.congress.gov'
params = {'api_key': congressapi}

bio_ids = usio_members['bioguide']
congress_members = pd.DataFrame()
i = 0
for b in bio_ids:
    time.sleep(5)
    if (i % 10)==0:
        print(f'Now working on member {i} of {len(bio_ids)}')
    try:
        if b != '':
            endpoint = f'/v3/member/{b}'
            r = requests.get(root + endpoint, params=params, headers=headers)
            myjson = json.loads(r.text)
            mem = pd.json_normalize(myjson['member'])
            congress_members = pd.concat([congress_members, mem])
    except:
        pass
    i += 1

Now working on member 0 of 539
Now working on member 10 of 539
Now working on member 20 of 539
Now working on member 30 of 539
Now working on member 40 of 539
Now working on member 50 of 539
Now working on member 60 of 539
Now working on member 70 of 539
Now working on member 80 of 539
Now working on member 90 of 539
Now working on member 100 of 539
Now working on member 110 of 539
Now working on member 120 of 539
Now working on member 130 of 539
Now working on member 140 of 539
Now working on member 150 of 539
Now working on member 160 of 539
Now working on member 170 of 539
Now working on member 180 of 539
Now working on member 190 of 539
Now working on member 200 of 539
Now working on member 210 of 539
Now working on member 220 of 539
Now working on member 230 of 539
Now working on member 240 of 539
Now working on member 250 of 539
Now working on member 260 of 539
Now working on member 270 of 539
Now working on member 280 of 539
Now working on member 290 of 539
Now working on member

In [7]:
congress_members.to_csv('Data/congress_members.csv', index=False)

## Congress API: sponsored legislation

In [8]:
congress_members = pd.read_csv('Data/congress_members.csv')

In [9]:
sponsor = pd.DataFrame()
params = {'api_key': congressapi,
         'limit': 250}
i = 1
l = len(congress_members['sponsoredLegislation.url'])
for u in congress_members['sponsoredLegislation.url']:
    time.sleep(5)
    print(f'Now compiling bills for member {i} of {l}')
    try:
        r = requests.get(url, headers=headers, params=params)
        myjson = json.loads(r.text)
        spons = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])
        spons = spons.query("congress==118")
        sponsor = pd.concat([sponsor, spons])
    except:
        pass
    i += 1

Now compiling bills for member 1 of 537
Now compiling bills for member 2 of 537
Now compiling bills for member 3 of 537
Now compiling bills for member 4 of 537
Now compiling bills for member 5 of 537
Now compiling bills for member 6 of 537
Now compiling bills for member 7 of 537
Now compiling bills for member 8 of 537
Now compiling bills for member 9 of 537
Now compiling bills for member 10 of 537
Now compiling bills for member 11 of 537
Now compiling bills for member 12 of 537
Now compiling bills for member 13 of 537
Now compiling bills for member 14 of 537
Now compiling bills for member 15 of 537
Now compiling bills for member 16 of 537
Now compiling bills for member 17 of 537
Now compiling bills for member 18 of 537
Now compiling bills for member 19 of 537
Now compiling bills for member 20 of 537
Now compiling bills for member 21 of 537
Now compiling bills for member 22 of 537
Now compiling bills for member 23 of 537
Now compiling bills for member 24 of 537
Now compiling bills for m

In [10]:
sponsor.to_csv('Data/bills.csv', index=False)

## Congress API: bills

## Congress API: committees/subcommittees

In [6]:
params = {'api_key': congressapi,
         'limit': 250,
         'offset': 0}
headers = {'User-Agent': useragent,
          'From': 'jkropko@virginia.edu'}
url = 'https://api.congress.gov/v3/committee'
r = requests.get(url, headers=headers,
                params=params)
myjson = json.loads(r.text)

In [9]:
myjson

{'error': "Reverse for 'committee_details' with keyword arguments '{'systemCode': 'LOC18200208', 'chamber': 'senate'}' not found. 2 pattern(s) tried: ['v3/committee/(?P<chamber>(house|senate|joint|House|Senate|Joint))/(?P<systemCode>([a-z0-9]+))(?P<file_ext>(\\\\.xml|\\\\.json|/))?$', 'v2/committee/(?P<chamber>(house|senate|joint|House|Senate|Joint))/(?P<systemCode>([a-z0-9]+))(?P<file_ext>(\\\\.xml|\\\\.json|/))?$'] (NoReverseMatch)",
 'request': {'contentType': 'application/json', 'format': 'json'}}

In [4]:
committees = pd.json_normalize(myjson, record_path=['committees'])
committees

NameError: name 'myjson' is not defined

In [ ]:
params = {'api_key': congressapi,
         'limit': 250,
         'offset': 250}
r = requests.get(url, headers=headers,
                params=params)
myjson = json.loads(r.text)
com2 = pd.json_normalize(myjson, record_path=['committees'])
committees = pd.concat([committees, com2])

In [ ]:
params = {'api_key': congressapi,
         'limit': 250,
         'offset': 500}
r = requests.get(url, headers=headers,
                params=params)
myjson = json.loads(r.text)
com2 = pd.json_normalize(myjson, record_path=['committees'])
committees = pd.concat([committees, com2])

In [ ]:
committees.to_csv("Data/committees.csv", index=False)

## https://theunitedstates.io/: committee assignments

In [17]:
url = 'https://theunitedstates.io/congress-legislators/committee-membership-current.json'
r = requests.get(url, headers=headers)
myjson = json.loads(r.text)
committee_members = pd.concat({k: pd.DataFrame(v) for k, v in myjson.items()}, axis=0).reset_index()
committee_members = committee_members[['level_0', 'rank', 'title', 'bioguide']]
committee_members = committee_members.rename({'level_0': 'committee_code'}, axis=1)
committee_members.to_csv('Data/committee_members.csv', index=False)

## OpenSecrets API: members

In [18]:
states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
legislators = pd.DataFrame()
for s in states:
    print(s)
    root = 'https://www.opensecrets.org/api/'
    headers = {'User-Agent': useragent,
              'From': 'jkropko@virginia.edu'}
    params = {'method': 'getLegislators',
              'id': s,
              'output': 'json',
              'apikey':opensecretsapi}
    r = requests.get(root, params=params, headers=headers)
    myjson = json.loads(r.text)
    leg = pd.json_normalize(myjson, record_path=['response', 'legislator'])
    legislators = pd.concat([legislators, leg])

AK
AL
AR
AZ
CA
CO
CT
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY


In [19]:
legislators.columns = legislators.columns.str.strip('@attributes.')
legislators.to_csv("Data/os_legislators.csv", index=False)

## OpenSecrets API: contributions

In [20]:
cid = [x for x in legislators['cid']]
root = 'https://www.opensecrets.org/api/'
headers = {'User-Agent': useragent,
          'From': 'jkropko@virginia.edu'}
contributions = pd.DataFrame()
i = 1
l = len(cid)
for c in cid:
    print(f'Now working on {i} of {l}')
    try:
        params = {'method': 'candContrib',
          'cid': c,
          'cycle':2022,
          'output': 'json',
          'apikey':opensecretsapi}
        r = requests.get(root, params=params, headers=headers)
        myjson = json.loads(r.text)
        contrib = pd.json_normalize(myjson, 
                                      record_path = ['response',
                                                     'contributors',
                                                     'contributor'])
        contributions = pd.concat([contributions, contrib])
    except:
        pass
    i += 1

Now working on 1 of 537
Now working on 2 of 537
Now working on 3 of 537
Now working on 4 of 537
Now working on 5 of 537
Now working on 6 of 537
Now working on 7 of 537
Now working on 8 of 537
Now working on 9 of 537
Now working on 10 of 537
Now working on 11 of 537
Now working on 12 of 537
Now working on 13 of 537
Now working on 14 of 537
Now working on 15 of 537
Now working on 16 of 537
Now working on 17 of 537
Now working on 18 of 537
Now working on 19 of 537
Now working on 20 of 537
Now working on 21 of 537
Now working on 22 of 537
Now working on 23 of 537
Now working on 24 of 537
Now working on 25 of 537
Now working on 26 of 537
Now working on 27 of 537
Now working on 28 of 537
Now working on 29 of 537
Now working on 30 of 537
Now working on 31 of 537
Now working on 32 of 537
Now working on 33 of 537
Now working on 34 of 537
Now working on 35 of 537
Now working on 36 of 537
Now working on 37 of 537
Now working on 38 of 537
Now working on 39 of 537
Now working on 40 of 537
Now worki

In [21]:
contributions.columns = contributions.columns.str.strip('@attributes.')
contributions.to_csv("Data/os_contributions.csv", index=False)

## Voteview: voting similarity matrix

In [2]:
url = 'https://voteview.com/static/data/out/votes/HS118_votes.csv'
votes = pd.read_csv(url)
votes = votes[['chamber','rollnumber','icpsr','cast_code']]
senate_votes = votes.query("chamber == 'Senate'")
paired_votes = pd.merge(senate_votes, senate_votes,
                       on = ['chamber','rollnumber'],
                       how = 'inner')
paired_votes = paired_votes.rename({'icpsr_x': 'icpsr',
                                   'icpsr_y': 'member_compare',
                                   'cast_code_x': 'vote',
                                   'cast_code_y': 'compare_vote'},
                                  axis=1)
paired_votes = paired_votes.query("icpsr != member_compare")
paired_votes = paired_votes.assign(
    equal_vote = paired_votes['vote'] == paired_votes['compare_vote']
)
paired_votes = paired_votes[['icpsr', 'member_compare', 'equal_vote']]
affinity = paired_votes.groupby(['icpsr', 'member_compare'])\
    .agg({'equal_vote': 'mean'}).reset_index()\
    .rename({'equal_vote':'affinity_score'}, axis=1)

In [3]:
affinity

,icpsr,member_compare,affinity_score
0,14226,14435,0.305147
1,14226,14858,0.375000
2,14226,14871,0.352941
3,14226,14921,0.772059
4,14226,15015,0.356618
...,...,...,...
10093,91300,42304,0.198529
10094,91300,42305,0.560000
10095,91300,49300,0.485597
10096,91300,49308,0.904412


In [4]:
house_votes = votes.query("chamber == 'House'")
maxrc = house_votes['rollnumber'].max()
house_votes

,chamber,rollnumber,icpsr,cast_code
0,House,1,14854,6
1,House,1,14863,6
2,House,1,14873,1
3,House,1,15029,1
4,House,1,15433,1
...,...,...,...,...
241429,House,553,31102,1
241430,House,553,31103,6
241431,House,553,39301,6
241432,House,553,39307,6


In [6]:
affinity2 = pd.DataFrame()
r = 0
while r < maxrc:
    print(r)
    hv = house_votes.query(f"rollnumber >= {r} & rollnumber < {r +100}")
    paired_votes = pd.merge(hv, hv,
                           on = ['chamber','rollnumber'],
                           how = 'inner')
    paired_votes = paired_votes.rename({'icpsr_x': 'icpsr',
                                       'icpsr_y': 'member_compare',
                                       'cast_code_x': 'vote',
                                       'cast_code_y': 'compare_vote'},
                                      axis=1)
    paired_votes = paired_votes.query("icpsr != member_compare")
    paired_votes = paired_votes.assign(
        equal_vote = paired_votes['vote'] == paired_votes['compare_vote']
    )
    paired_votes = paired_votes[['icpsr', 'member_compare', 'equal_vote']]
    a2 = paired_votes.groupby(['icpsr', 'member_compare'])\
        .agg({'equal_vote': ['sum', 'count']}).reset_index()\
        .rename({'equal_vote':'affinity_score'}, axis=1)
    affinity2 = pd.concat([affinity2, a2])
    r += 100

0
100
200
300
400
500


In [7]:
affinity2.columns = ['icpsr', 'member_compare', 'total_agree', 'total_vote'] 
affinity2 = affinity2.groupby(['icpsr','member_compare'])\
    .agg({'total_agree': 'sum', 'total_vote':'sum'}).reset_index()

In [8]:
affinity2['affinity_score'] = affinity2['total_agree']/affinity2['total_vote']
affinity2 = affinity2.drop(['total_agree','total_vote'], axis=1)
affinity = pd.concat([affinity, affinity2])
affinity.to_csv('Data/votingaffinity.csv', index=False)

## Voteview: ideology

In [ ]:
url = 'https://voteview.com/static/data/out/members/HS118_members.csv'
ideology = pd.read_csv(url)
ideology = to_csv("Data/ideology.csv", index=False)

## Cleaning the columns

In [19]:
bills=pd.read_csv("Data/bills.csv")
bills.columns = [x.lower() for x in bills.columns]
bills.columns = [x.replace('.', '_') for x in bills.columns]
bills.columns

Index(['congress', 'introduceddate', 'number', 'title', 'type', 'url',
       'latestaction_actiondate', 'latestaction_text', 'policyarea_name',
       'amendmentnumber', 'latestaction', 'latestaction_actiontime'],
      dtype='object')

In [20]:
congress_members=pd.read_csv("Data/congress_members.csv") 
congress_members.columns = [x.lower() for x in congress_members.columns]
congress_members.columns = [x.replace('.', '_') for x in congress_members.columns]
congress_members = congress_members.drop(['partyhistory', 'updatedate',
                                         'depiction_attribution',
                                         'middlename', 'suffixname', 'nickname'], 
                                         axis=1)
congress_members.columns

Index(['bioguideid', 'birthyear', 'currentmember', 'directordername',
       'firstname', 'honorificname', 'invertedordername', 'lastname',
       'officialwebsiteurl', 'partyhistory', 'state', 'terms', 'updatedate',
       'addressinformation_city', 'addressinformation_district',
       'addressinformation_officeaddress', 'addressinformation_phonenumber',
       'addressinformation_zipcode', 'cosponsoredlegislation_count',
       'cosponsoredlegislation_url', 'depiction_attribution',
       'depiction_imageurl', 'sponsoredlegislation_count',
       'sponsoredlegislation_url', 'middlename', 'suffixname', 'nickname',
       'leadership', 'district'],
      dtype='object')

In [77]:
lead = congress_members[~congress_members['leadership'].isna()].reset_index()
z = zip(lead['bioguideid'], lead['leadership'])
def buildlead(m, l):
    leaddf = pd.DataFrame(eval(l))
    leaddf['bioguideid'] = m
    return leaddf
leadlist = [buildlead(m, l) for m, l in z]
leadership = pd.concat(leadlist)
leadership.to_csv('Data/leadership.csv', index=False)

In [79]:
z = zip(lead['bioguideid'], lead['terms'])
def buildterms(m, t):
    termsdf = pd.DataFrame(eval(t))
    termsdf['bioguideid'] = m
    return termsdf
termslist = [buildterms(m, l) for m, l in z]
terms = pd.concat(termslist)
terms.to_csv('Data/terms.csv', index=False)

In [21]:
os_legislators=pd.read_csv("Data/os_legislators.csv")
os_legislators.columns = [x.lower() for x in os_legislators.columns]
os_legislators.columns = [x.replace('.', '_') for x in os_legislators.columns]
os_legislators = os_legislators.drop(['firstl', 'lastnam',], axis=1)
os_legislators = os_legislators.rename({'cid', 
                                        'party', 
                                        'offic', 'gend', 'first_elected',
       'xit_cod', 'commen', 'phon', 'fax', 'w', 'webform', 'congress_offic',
       'oguide_id', 'votesmart_id', 'feccandid', 'witter_id', 'youtube_url',
       'facebook_id', 'hd'})
os_legislators.columns

Index(['cid', 'firstl', 'lastnam', 'party', 'offic', 'gend', 'first_elected',
       'xit_cod', 'commen', 'phon', 'fax', 'w', 'webform', 'congress_offic',
       'oguide_id', 'votesmart_id', 'feccandid', 'witter_id', 'youtube_url',
       'facebook_id', 'hd'],
      dtype='object')

In [22]:
committee_members=pd.read_csv("Data/committee_members.csv") 
committee_members.columns = [x.lower() for x in committee_members.columns]
committee_members.columns = [x.replace('.', '_') for x in committee_members.columns]
committee_members.columns

Index(['committee_code', 'rank', 'title', 'bioguide'], dtype='object')

In [23]:
committees=pd.read_csv("Data/committees.csv")
committees.columns = [x.lower() for x in committees.columns]
committees.columns = [x.replace('.', '_') for x in committees.columns]
committees.columns

Index(['chamber', 'committeetypecode', 'name', 'systemcode', 'url',
       'parent_name', 'parent_systemcode', 'parent_url', 'subcommittees'],
      dtype='object')

In [24]:
os_contributions=pd.read_csv("Data/os_contributions.csv")
os_contributions.columns = [x.lower() for x in os_contributions.columns]
os_contributions.columns = [x.replace('.', '_') for x in os_contributions.columns]
os_contributions.columns

Index(['org_nam', 'otal', 'pac', 'ndiv'], dtype='object')

In [25]:
votingaffinity=pd.read_csv("Data/votingaffinity.csv")
votingaffinity.columns = [x.lower() for x in votingaffinity.columns]
votingaffinity.columns = [x.replace('.', '_') for x in votingaffinity.columns]
votingaffinity.columns

Index(['icpsr', 'member_compare', 'affinity_score'], dtype='object')